In [299]:
import redis,json
conPool=redis.ConnectionPool(host='localhost',port=6379,db=0,encoding='utf-8',decode_responses=True)
connections=redis.Redis(connection_pool=conPool)
from dateutil.parser import parse


class RedObject(object):
    def __init__(self,**kwargs):
        self.__dict__.update(kwargs)
        if not 'key' in locals()['kwargs'].keys() or not 'filterBy' in locals()['kwargs'].keys():
            self.__meta__=json.loads(self.redis.hget(self.collection,'__meta__'))
            self.key=self.__meta__['key']
            self.filterBy=self.__meta__['filterBy']
        self.__meta__={"key":self.key,"filterBy":self.filterBy}
        self.redis.hset(self.collection,'__meta__',json.dumps(self.__meta__))
            
    def __repr__(self):
        return self.collection
    
    def getMessage(self,key):
        msg=self.redis.hget(self.collection,key)
        if not msg:
            return {}
        return json.loads(msg)
    
    def getAllMessage(self):
        msg=self.redis.hgetall(self.collection)
        return [json.loads(x) for j,x in enumerate(msg.values()) if list(msg.keys())[j] != '__meta__']
    
    def pushMessage(self,msg):
#         msg['host']=self.host
        pMsg=self.getMessage(msg[self.key])
        if not pMsg:
            dt=parse(msg[self.filterBy],tzinfos=timezone_info)
            prev=self.redis.hmget(self.collection+'_lookup',[dt.strftime('%s')])
            if  prev[0]:
                prev=prev[0]+'|'+msg[self.key]
            else:
                prev=msg[self.key]
            self.redis.hmset(self.collection+'_lookup',{dt.strftime('%s'):prev})
            msg['Start']=msg.get(self.filterBy)
            self.redis.hset(self.collection,msg[self.key],json.dumps(msg))

        else:
            if not pMsg.get('Start'):
                pMsg['Start']=pMsg.get(self.filterBy)
            else:
                if not pMsg.get('End'):
                    pMsg['End']=msg.get(self.filterBy)
            for i,j in msg.items():
                pMsg[i]=j
            self.redis.hset(self.collection,msg[self.key],json.dumps(pMsg))
        return None
    
    def pusMessages(self,keys,messages):
        return [self.pushMessage(keys[i],msg) for i, msg in enumerate(messages) ]
    
    def FilterBy(self,keys):
        keys=self.redis.hmget(self.collection+'_lookup',keys)
        print(keys)
        if len(keys)>0:
            key=""
            for j in keys:
                if j:
                    key=key+'|'+j
            key=key.split('|')
            print(key)
            return [self.getMessage(key1) for key1 in key if key1]
        return []
    
s=RedObject(collection='Schedules',key='Name',filterBy='Timestamp',redis=connections)
# s.pushMessage('Firstkey',{"Name":"Arun","Age":100})

In [301]:
s.pushMessage({"Name":"Ajith","Age":101,"Timestamp":'Thu 02 Apr 2020 17:11:11 AM +08'})
s.FilterBy([1585757471,1585757472])


['Singam|Arun|Ajith', None]
['', 'Singam', 'Arun', 'Ajith']


[{'Name': 'Singam',
  'Age': 101,
  'Timestamp': 'Thu 02 Apr 2020 12:11:11 AM +08'},
 {'Name': 'Arun',
  'Age': 101,
  'Timestamp': 'Thu 02 Apr 2020 12:11:11 AM +08',
  'Start': 'Thu 02 Apr 2020 12:11:11 AM +08',
  'End': 'Thu 02 Apr 2020 12:11:11 AM +08'},
 {'Name': 'Ajith',
  'Age': 101,
  'Timestamp': 'Thu 02 Apr 2020 17:11:11 AM +08',
  'Start': 'Thu 02 Apr 2020 12:11:11 AM +08',
  'End': 'Thu 02 Apr 2020 17:11:11 AM +08'}]

In [163]:
timezone_info = {"A":"UTC +1","ACDT":"UTC +10:30","ACST":"UTC +9:30","ACT":"UTC -5","ACWST":"UTC +8:45","ADT":"UTC +4","AEDT":"UTC +11","AEST":"UTC +10","AET":"UTC +10:00 / +11:00","AFT":"UTC +4:30","AKDT":"UTC -8","AKST":"UTC -9","ALMT":"UTC +6","AMST":"UTC -3","AMT":"UTC -4","ANAST":"UTC +12","ANAT":"UTC +12","AQTT":"UTC +5","ART":"UTC -3","AST":"UTC +3","AT":"UTC -4:00 / -3:00","AWDT":"UTC +9","AWST":"UTC +8","AZOST":"UTC +0","AZOT":"UTC -1","AZST":"UTC +5","AZT":"UTC +4","AoE":"UTC -12","B":"UTC +2","BNT":"UTC +8","BOT":"UTC -4","BRST":"UTC -2","BRT":"UTC -3","BST":"UTC +6","BTT":"UTC +6","C":"UTC +3","CAST":"UTC +8","CAT":"UTC +2","CCT":"UTC +6:30","CDT":"UTC -5","CEST":"UTC +2","CET":"UTC +1","CHADT":"UTC +13:45","CHAST":"UTC +12:45","CHOST":"UTC +9","CHOT":"UTC +8","CHUT":"UTC +10","CIDST":"UTC -4","CIST":"UTC -5","CKT":"UTC -10","CLST":"UTC -3","CLT":"UTC -4","COT":"UTC -5","CST":"UTC -6","CT":"UTC -6:00 / -5:00","CVT":"UTC -1","CXT":"UTC +7","ChST":"UTC +10","D":"UTC +4","DAVT":"UTC +7","DDUT":"UTC +10","E":"UTC +5","EASST":"UTC -5","EAST":"UTC -6","EAT":"UTC +3","ECT":"UTC -5","EDT":"UTC -4","EEST":"UTC +3","EET":"UTC +2","EGST":"UTC +0","EGT":"UTC -1","EST":"UTC -5","ET":"UTC -5:00 / -4:00","F":"UTC +6","FET":"UTC +3","FJST":"UTC +13","FJT":"UTC +12","FKST":"UTC -3","FKT":"UTC -4","FNT":"UTC -2","G":"UTC +7","GALT":"UTC -6","GAMT":"UTC -9","GET":"UTC +4","GFT":"UTC -3","GILT":"UTC +12","GMT":"UTC +0","GST":"UTC +4","GYT":"UTC -4","H":"UTC +8","HDT":"UTC -9","HKT":"UTC +8","HOVST":"UTC +8","HOVT":"UTC +7","HST":"UTC -10","I":"UTC +9","ICT":"UTC +7","IDT":"UTC +3","IOT":"UTC +6","IRDT":"UTC +4:30","IRKST":"UTC +9","IRKT":"UTC +8","IRST":"UTC +3:30","IST":"UTC +5:30","JST":"UTC +9","K":"UTC +10","KGT":"UTC +6","KOST":"UTC +11","KRAST":"UTC +8","KRAT":"UTC +7","KST":"UTC +9","KUYT":"UTC +4","L":"UTC +11","LHDT":"UTC +11","LHST":"UTC +10:30","LINT":"UTC +14","M":"UTC +12","MAGST":"UTC +12","MAGT":"UTC +11","MART":"UTC -9:30","MAWT":"UTC +5","MDT":"UTC -6","MHT":"UTC +12","MMT":"UTC +6:30","MSD":"UTC +4","MSK":"UTC +3","MST":"UTC -7","MT":"UTC -7:00 / -6:00","MUT":"UTC +4","MVT":"UTC +5","MYT":"UTC +8","N":"UTC -1","NCT":"UTC +11","NDT":"UTC -2:30","NFT":"UTC +11","NOVST":"UTC +7","NOVT":"UTC +7","NPT":"UTC +5:45","NRT":"UTC +12","NST":"UTC -3:30","NUT":"UTC -11","NZDT":"UTC +13","NZST":"UTC +12","O":"UTC -2","OMSST":"UTC +7","OMST":"UTC +6","ORAT":"UTC +5","P":"UTC -3","PDT":"UTC -7","PET":"UTC -5","PETST":"UTC +12","PETT":"UTC +12","PGT":"UTC +10","PHOT":"UTC +13","PHT":"UTC +8","PKT":"UTC +5","PMDT":"UTC -2","PMST":"UTC -3","PONT":"UTC +11","PST":"UTC -8","PT":"UTC -8:00 / -7:00","PWT":"UTC +9","PYST":"UTC -3","PYT":"UTC -4","Q":"UTC -4","QYZT":"UTC +6","R":"UTC -5","RET":"UTC +4","ROTT":"UTC -3","S":"UTC -6","SAKT":"UTC +11","SAMT":"UTC +4","SAST":"UTC +2","SBT":"UTC +11","SCT":"UTC +4","SGT":"UTC +8","SRET":"UTC +11","SRT":"UTC -3","SST":"UTC -11","SYOT":"UTC +3","T":"UTC -7","TAHT":"UTC -10","TFT":"UTC +5","TJT":"UTC +5","TKT":"UTC +13","TLT":"UTC +9","TMT":"UTC +5","TOST":"UTC +14","TOT":"UTC +13","TRT":"UTC +3","TVT":"UTC +12","U":"UTC -8","ULAST":"UTC +9","ULAT":"UTC +8","UTC":"UTC","UYST":"UTC -2","UYT":"UTC -3","UZT":"UTC +5","V":"UTC -9","VET":"UTC -4","VLAST":"UTC +11","VLAT":"UTC +10","VOST":"UTC +6","VUT":"UTC +11","W":"UTC -10","WAKT":"UTC +12","WARST":"UTC -3","WAST":"UTC +2","WAT":"UTC +1","WEST":"UTC +1","WET":"UTC +0","WFT":"UTC +12","WGST":"UTC -2","WGT":"UTC -3","WIB":"UTC +7","WIT":"UTC +9","WITA":"UTC +8","WST":"UTC +14","WT":"UTC +0","X":"UTC -11","Y":"UTC -12","YAKST":"UTC +10","YAKT":"UTC +9","YAPT":"UTC +10","YEKST":"UTC +6","YEKT":"UTC +5","Z":"UTC +0"}
import datetime
dt=parse('Thu 02 Apr 2020 12:11:10 AM +08',tzinfos=timezone_info)
dt.strftime('%s')

'1585757470'

In [167]:
connections.hmget

<bound method Redis.hmget of Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>>